In [ ]:
%matplotlib inline

## YOLO

In [ ]:
from src.hpe.common.mutators import convert_to_bgr
from src.hpe.yolo.performance import DistanceCollector

In [ ]:
# default estimation
collector = DistanceCollector()
distances = collector.collect(name="distances")

In [ ]:
# estimation on bgr images
distances_bgr = collector.collect(
    name="distances_bgr",
    image_mutators=[convert_to_bgr]
)

In [1]:
from src.hpe.yolo.performance import PerformanceLogger

loaded labels


In [2]:
logger = PerformanceLogger()
logger.collect(name="Yolov11m-pose")


0: 448x640 1 person, 39.2ms
Speed: 29.1ms preprocess, 39.2ms inference, 928.8ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 person, 41.1ms
Speed: 4.2ms preprocess, 41.1ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 448)

0: 448x640 2 persons, 16.9ms
Speed: 3.4ms preprocess, 16.9ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 (no detections), 17.0ms
Speed: 2.7ms preprocess, 17.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 448)

0: 448x640 1 person, 15.9ms
Speed: 2.6ms preprocess, 15.9ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 person, 16.6ms
Speed: 2.9ms preprocess, 16.6ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 448)

0: 448x640 1 person, 16.0ms
Speed: 2.6ms preprocess, 16.0ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 person, 17.4ms
Speed: 2.7ms preprocess, 17.4ms inference, 4.1ms postprocess per image at shap

In [ ]:
# calculate and log performance on bgr images
logger.collect(
    name="Yolov11m-pose on bgr",
    image_mutators=[convert_to_bgr]
)

In [ ]:
from src.hpe.yolo.performance import read_distances

distances = read_distances()
distances_bgr = read_distances(dataset_name="distances_bgr")

In [ ]:
from src.hpe.yolo.plot import plot_yolo_average_distances

In [ ]:
plot_yolo_average_distances(distances=distances)

In [ ]:

plot_yolo_average_distances(distances=distances_bgr)

In [ ]:
from src.hpe.yolo.play import play_with_hpe as play_with_yolo_hpe
from glob import glob
from random import choice

video_paths = glob("data/samples/**/*.*")
video_path = choice(video_paths)

play_with_yolo_hpe(video_path)

## MediaPipe

In [ ]:
from src.hpe.mp.performance import DistanceCollector
from src.hpe.common.mutators import convert_to_bgr

In [ ]:
collector = DistanceCollector()
distances = collector.collect(name="distances")

In [ ]:
# estimation on bgr images
distances_bgr = collector.collect(
    name="distances_bgr",
    image_mutators=[convert_to_bgr]
)

In [ ]:
from src.hpe.mp.performance import PerformanceLogger

In [ ]:
logger = PerformanceLogger()
logger.collect(name="MediaPipe")

In [ ]:
logger.collect(name="MediaPipe on bgr",
    image_mutators=[convert_to_bgr])

In [ ]:
from src.hpe.mp.performance import read_distances
distances = read_distances()
distances_bgr = read_distances(dataset_name="distances_bgr")

In [ ]:
from src.hpe.mp.plot import plot_mediapipe_average_distances

In [ ]:

plot_mediapipe_average_distances(distances=distances)

In [ ]:
plot_mediapipe_average_distances(distances=distances_bgr)

In [ ]:
from src.hpe.mp.play import play_with_hpe
from glob import glob
from random import choice

video_paths = glob("data/samples/**/*.*")
video_path = choice(video_paths)

play_with_hpe(video_path)

## Comparison

In [ ]:
from src.hpe.yolo.performance import read_distances as read_yolo_distances
from src.hpe.mp.performance import read_distances as read_mp_distances
from src.hpe.common.plot import plot_distances_boxplot

In [ ]:
mp_dist = read_mp_distances(dataset_name="distances")
mp_bgr_dist = read_mp_distances(dataset_name="distances_bgr")

yolo_dist = read_yolo_distances(dataset_name="distances")
yolo_bgr_dist = read_yolo_distances(dataset_name="distances_bgr")


In [ ]:
plot_distances_boxplot(
    None,
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
plot_distances_boxplot(
    (0, 10),
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))